In [ ]:
# This notebook exemplifies how to calculate compound hazard metrics and plot their monthly patterns

In [ ]:
import pandas as pd
import xarray as xr
# Read all chosen indicators to represent hazards and plot seasonal patterns for single and compound events
hwi= xr.open_dataset('heatwave_SMHI.nc')
spei12_or= xr.open_dataset('SPEI12.nc')
spei12 = spei12_or.resample(time='D').pad()
# Perform linear interpolation to convert monthly data to daily data
spei12 = spei12.interpolate_na(dim='time', method='linear')
dfi= xr.open_dataset('Fidx.nc')



In [ ]:
# Convert 'time' dtype to datetime64[ns]
dfi['time'] = xr.DataArray(pd.to_datetime(dfi.time.values))

import numpy as np

# Load the data into memory
dfi['flood_index'].load()
# Convert the time variable to a Pandas datetime index
dfi['time'] = pd.DatetimeIndex(dfi['time'].values)

# Extract the flood_index variable as a DataArray
dfi_flood_index = dfi['flood_index']

start_date = pd.to_datetime('1922-01-31')
end_date = pd.to_datetime('2021-12-31')

# Truncate hwi array
hwi = hwi.sel(time=slice(start_date, end_date))

# Truncate spei12 array
spei12= spei12.sel(time=slice(start_date, end_date))

# Truncate dfi array
dfi = dfi.sel(time=slice(start_date, end_date))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming dfi is your DataFrame and flood_index is a column containing the data
data = hwi['heatwave index'][:, 30, 30].values

# Plotting the histogram
plt.hist(data, bins='auto', color='blue', alpha=0.7)

# Adding labels and title
plt.xlabel('Heatwave Index')
plt.ylabel('Frequency')
plt.title('Histogram of HWI')

# Display the histogram
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming dfi is your DataFrame and flood_index is a column containing the data
#data = spei12_or.SPEI[:, 30, 30].values # without interpolation
data = spei12.SPEI[:, 30, 30].values # linearly interpolated

# Plotting the histogram
plt.hist(data, bins='auto', color='blue', alpha=0.7)

# Adding labels and title
plt.xlabel('Drought Index')
plt.ylabel('Frequency')
plt.title('Histogram of SPEI12')

# Display the histogram
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming dfi is your DataFrame and flood_index is a column containing the data
data = dfi.flood_index[:, 30, 30].values

# Plotting the histogram
plt.hist(data, bins='auto', color='blue', alpha=0.7)

# Adding labels and title
plt.xlabel('Flood Index')
plt.ylabel('Frequency')
plt.title('Histogram of Flood Index')

# Display the histogram
plt.show()


In [ ]:
import numpy as np

def generate_bootstrapped_timeseries(spei12_series):
    num_days = len(spei12_series)
    #resampled_timeseries = np.empty((num_resamples, num_days))
    # Generate random indices with replacement
    random_indices = np.random.randint(0, num_days, size=num_days)
    # Create the resampled timeseries using the random indices
    resampled_timeseries = spei12_series[random_indices]
    return resampled_timeseries
# Calculate joint return period from joint probability of exceedance of two co-occuring hazards
# Formulation from 'Global hotspots for the occurence of compound events'
import numpy as np
def rp(hwi_array, spei12_array, hwi_t, spei12_t, hwi_type, spei12_type):
    rp_array = np.empty((hwi_array.shape[1], hwi_array.shape[2]))
    rp_array[:] = np.nan
    ns_array = rp_array.copy() # not statistically significant cells
    with tqdm(total=hwi_array.shape[1]) as pbar:
        for i in range(hwi_array.shape[1]):
            #print(i)
            for j in range(hwi_array.shape[2]):
                if np.isnan(hwi_array[:, i, j]).any():
                    continue
                else:
                    hwi_series = hwi_array[:, i, j].copy()
                    spei12_series = spei12_array[:, i, j].copy()
                    # 0: no event, 1: event happens
                    if hwi_type == 'lower':
                        hwi_series[hwi_series >= hwi_t] = 1
                        hwi_series[hwi_series < hwi_t] = 0
                    else:
                        hwi_series[hwi_series > hwi_t] = 0
                        hwi_series[hwi_series <= hwi_t] = 1
                    if spei12_type == 'lower':
                        spei12_series[spei12_series >= spei12_t] = 1
                        spei12_series[spei12_series < spei12_t] = 0
                    else:
                        spei12_series[spei12_series > spei12_t] = 0
                        spei12_series[spei12_series <= spei12_t] = 1
                    # Create the joint_series by performing logical AND operation
                    joint_series = np.array(hwi_series, dtype=int) & np.array(spei12_series, dtype=int)
                    #print(joint_series)
                    if np.nansum(joint_series)>0:
                        # Calculate the daily probability of joint exceedance for a given month
                        w = np.nansum(joint_series) # number of daily events in 100 years
                        w_year= w/100 # average number of daily events in 1 year 
                        pjoint_exc = w_year / (365.25/12) # yearly probability of exceedance
                        # Perform the significance test
                        p = 0
                        for N in range(1,1001):
                            spei12_series_test = generate_bootstrapped_timeseries(spei12_series)
                            joint_series_test = np.array(hwi_series, dtype=int) & np.array(spei12_series_test, dtype=int)
                            ws = np.nansum(joint_series_test)
                            n = np.random.uniform(-0.0009, 0.0009)
                            ws_dis = ws + n
                            if ws_dis>w: # check if disturbed sum is larger than original sum
                                p = p+1
                        #p = 10000 # skip test
                        p_value= p/1000
                        #rp_point = 1 / (pjoint_exc * (len(joint_series) / 100))
                        rp_point = 1 / pjoint_exc
                        rp_array[i, j] = rp_point
                        # Check if it is significant
                        if p_value < 0.05:  # 0.05 for 5% significance
                            ns_array[i, j] = 1
                            print('reject the null hypothesis that the joint exceedance between hazard X and hazard Y was introduced by random chance, not statistally significant at 5%')
            pbar.update()
    return rp_array, ns_array


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from cartopy.io import shapereader
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
import geopandas as gpd
from tqdm.notebook import tqdm
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
def remove_leap_years(input_array, start_year):
    years = np.arange(start_year, start_year + 100)
    # Define the list of leap years
    leap_years_list = [1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
    # Find the indices corresponding to leap years in the years array
    leap_year_indices = np.where(np.isin(years, leap_years_list))[0]
    # Initialize the output array to store data without leap years
    output_array = []
    idx_start = 0
    idx_end = 28
    for i, year in enumerate(years):
        # Skip data for leap years
        output_array.append(input_array[idx_start:idx_end,:])
        if i in leap_year_indices:
            idx_start += 29
            idx_end += 29
        else:
            idx_start += 28
            idx_end += 28
    # Convert the list of arrays to a single NumPy array with shape (2800, 100, 100)
    output_array = np.concatenate(output_array, axis=0)
    return output_array
def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]

def plot_rp(hazard, hwi, spei12, dfi):
    sy= int(hwi.time.values[0].astype('M8[Y]').astype(str))
    label = 'Joint return period (years): ' + hazard
    # Define the custom colorbar intervals
    custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500]
    #[1, 1.5, 2, 5, 10, 15, 20, 30, 50, 75, 100]
    #[1, 2, 5, 10, 25, 50, 75, 100, 200, 500]
    # Set the boundaries for the colorbar intervals using BoundaryNorm
    cmap = cm.jet_r
    norm = mcolors.BoundaryNorm(custom_ticks, cmap.N, extend='both')
    #cmap = cm.get_cmap(cmap, len(custom_ticks) - 1)
    # Define a grayscale colormap
    cmap_colors = [(0.5, 0.5, 0.5), (0.5, 0.5, 0.5)]  # Grey color for both values
    gray_cmap = ListedColormap(cmap_colors, name='CustomGrey')
    months = range(1, 13)
    # Set up the figure and axes
    fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(16, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    # Flatten the axes array for easier iteration
    axes = axes.flatten()
    # Loop over the axes and customize each subplot
    with tqdm(total=len(months)) as pbar:
        for i, ax in enumerate(axes):
            month = months[i]
            # Select data for the specific month
            hwi_month = hwi.sel(time=hwi['time'].dt.month == month)
            spei12_month = spei12.sel(time=spei12['time'].dt.month == month)
            dfi_month = dfi_flood_index.where(dfi['time'].dt.month == month, drop=True)
            hwi_array = hwi_month['heatwave index'].values
            spei12_array = spei12_month['SPEI'].values
            dfi_array = dfi_month.values
            if i==0:
                # Create a grid of longitudes and latitudes
                lon_grid, lat_grid = np.meshgrid(hwi_month['lon'], hwi_month['lat'])
                # Add the country boundaries
                resolution = '10m'
                category = 'cultural'
                name = 'admin_0_countries'
                shpfilename = shapereader.natural_earth(resolution, category, name)
                df = gpd.read_file(shpfilename)
                # get geometry of a country
                poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]
                stamen_terrain = cimgt.Stamen('terrain-background')
                pad1 = .1  # padding, degrees unit
                exts = [poly[0].bounds[0] - pad1, poly[0].bounds[2] + pad1, poly[0].bounds[1] - pad1,
                        poly[0].bounds[3] + pad1];
                # make a mask polygon by polygon's difference operation
                msk = Polygon(rect_from_bound(*exts)).difference(poly[0].simplify(0.01))
                msk_stm = ccrs.PlateCarree().project_geometry(msk, ccrs.PlateCarree())  # project geometry to the projection used by stamen
            ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black')
            ax.set_extent(exts, crs=ccrs.PlateCarree())
            # plot the mask using semi-transparency (alpha=0.65) on the masked-out portion
            ax.add_geometries(msk_stm, ccrs.PlateCarree(), zorder=12, facecolor='white', edgecolor='k', alpha=1)
            # Plot the contour lines for the correlation between Hazards
            if hazard[0:21] == 'heat wave and drought':
                rp_array, ns_array=rp(hwi_array, spei12_array, 0, -1, 'lower', 'upper')
                im = ax.imshow(rp_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, norm=norm)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:19] == 'heat wave and flood':
                if month==2:
                    hwi_array = remove_leap_years(hwi_array, sy)
                rp_array, ns_array=rp(hwi_array, dfi_array, 0, 0, 'lower', 'lower')
                im = ax.imshow(rp_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, norm=norm)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:17] == 'drought and flood':
                if month==2:
                    spei12_array = remove_leap_years(spei12_array, sy)
                rp_array, ns_array=rp(dfi_array, spei12_array, 0, -1, 'lower', 'upper')
                im = ax.imshow(rp_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, norm=norm)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard == 'heat wave' or hazard == 'drought' or hazard == 'flood':
                label = 'Return period (years): ' + hazard
                threshold_single=0
                if hazard=='heat wave':
                    haz_array=hwi_array
                    haz_type='lower'
                elif hazard=='drought':
                    #spei12_month = spei12_or.sel(time=spei12_or['time'].dt.month == month)
                    #spei12_array = spei12_month['SPEI'].values
                    #custom_ticks = [0.0001, 0.001, 0.01, 0.1, 0.5, 1]
                    #norm = mcolors.BoundaryNorm(custom_ticks, cmap.N, extend='both')
                    haz_array=spei12_array
                    haz_type='upper'
                    threshold_single= -1
                elif hazard=='flood':
                    haz_array=dfi_array
                    haz_type='lower'
                rp_array, ns_array=rp_single(haz_array, threshold_single, haz_type)
                im = ax.imshow(rp_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, norm=norm)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            # Add a title
            month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                          'October', 'November', 'December']
            ax.set_title(month_list[month - 1], fontsize=20)
            pbar.update()
    cbar_ax = fig.add_axes([0, 0, 1, 0.05])  # [left, bottom, width, height]
    # Create the colorbar using the custom ticks and boundaries
    # Create a custom colormap using ListedColormap
    #colors = plt.cm.coolwarm_r(norm(np.arange(len(custom_ticks) - 1)))
    cbar = plt.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both',
                        ticks=custom_ticks, boundaries=custom_ticks, norm=norm, cmap=cmap)
    cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label, fontsize=20)
    plt.subplots_adjust(wspace=0.2, hspace=-0.2)  # Adjust the value of hspace as needed to reduce the space between rows
    plt.tight_layout()
    # Show the plot
    plt.show()


In [ ]:
plot_rp('heat wave and drought', hwi, spei12, dfi, 'jet_r')


In [ ]:
plot_rp('heat wave and flood', hwi, spei12, dfi, 'jet_r')

In [ ]:
plot_rp('drought and flood', hwi, spei12, dfi, 'jet_r') 

In [ ]:
# Calculate joint return period from joint probability of exceedance of two co-occuring hazards
# Formulation from 'Global hotspots for the occurence of compound events'
import numpy as np
def rp_single(hwi_array, hwi_t, hwi_type):
    rp_array = np.empty((hwi_array.shape[1], hwi_array.shape[2]))
    rp_array[:] = np.nan
    ns_array = rp_array.copy() # not statistically significant cells
    with tqdm(total=hwi_array.shape[1]) as pbar:
        for i in range(hwi_array.shape[1]):
            for j in range(hwi_array.shape[2]):
                hwi_series = hwi_array[:, i, j].copy()
                # 0: no event, 1: event happens
                if hwi_type == 'lower':
                    hwi_series[hwi_series >= hwi_t] = 1
                    hwi_series[hwi_series < hwi_t] = 0
                else:
                    #print('upper type for droughts')
                    hwi_series[hwi_series > hwi_t] = 0
                    hwi_series[hwi_series <= hwi_t] = 1
                
                # Calculate the daily probability of joint exceedance for a given month
                w = np.nansum(hwi_series)
                w_year= w/100 # average number of daily events in 1 year 
                p_exc = w_year / (365.25/12) # yearly probability of exceedance
                # Perform the significance test
                p = 0
                for N in range(1,1001):
                    hwi_series_test = generate_bootstrapped_timeseries(hwi_series)
                    ws = np.nansum(hwi_series_test)
                    n = np.random.uniform(-0.0009, 0.0009)
                    ws_dis = ws + n
                    if ws_dis>w: # check if disturbed sum is larger than original sum
                        p = p+1
                p_value= p/1000
                rp_point = 1 / p_exc # return period in years
                rp_array[i, j] = rp_point
                # Check if it is significant
                if p_value < 0.05:  # 0.05 for 5% significance
                    ns_array[i, j] = 1
                    print('reject the null hypothesis that the joint exceedance between hazard X and hazard Y was introduced by random chance, not statistally significant at 5%')
            pbar.update()
    return rp_array, ns_array


In [ ]:
plot_rp('heat wave', hwi, spei12, dfi, 'jet_r')

In [ ]:
plot_rp('drought', hwi, spei12, dfi, 'jet_r')

In [ ]:
plot_rp('flood', hwi, spei12, dfi, 'jet_r')

In [ ]:
# Calculate Likelihood Multiplication Factor to check for independence and correlations
# Formulation from 'Global hotspots for the occurence of compound events'
import numpy as np
def lmf(hwi_array, spei12_array, hwi_t, spei12_t, hwi_type, spei12_type):
    lmf_array = np.empty((hwi_array.shape[1], hwi_array.shape[2]))
    lmf_array[:] = np.nan
    ns_array = lmf_array.copy() # not statistically significant cells
    ns_array[:] = np.nan
    with tqdm(total=hwi_array.shape[1]) as pbar:
        for i in range(hwi_array.shape[1]):
            #print(i)
            for j in range(hwi_array.shape[2]):
                if np.isnan(hwi_array[:, i, j]).any():
                    continue
                else:
                    hwi_series = hwi_array[:, i, j].copy()
                    spei12_series = spei12_array[:, i, j].copy()
                    # 0: no event, 1: event happens
                    if hwi_type == 'lower':
                        hwi_series[hwi_series >= hwi_t] = 1
                        hwi_series[hwi_series < hwi_t] = 0
                    else:
                        hwi_series[hwi_series > hwi_t] = 0
                        hwi_series[hwi_series <= hwi_t] = 1
                    if spei12_type == 'lower':
                        spei12_series[spei12_series >= spei12_t] = 1
                        spei12_series[spei12_series < spei12_t] = 0
                    else:
                        spei12_series[spei12_series > spei12_t] = 0
                        spei12_series[spei12_series <= spei12_t] = 1
                    # Create the joint_series by performing logical AND operation
                    joint_series = np.array(hwi_series, dtype=int) & np.array(spei12_series, dtype=int)
                    if np.nansum(joint_series)>0:
                        # Calculate the daily probability of joint exceedance for a given month
                        w = np.nansum(joint_series)
                        pactual = w / len(joint_series)
                        # Perform the significance test
                        p = 0
                        for N in range(1,1001):
                            spei12_series_test = generate_bootstrapped_timeseries(spei12_series)
                            joint_series_test = np.array(hwi_series, dtype=int) & np.array(spei12_series_test, dtype=int)
                            ws = np.nansum(joint_series_test)
                            n = np.random.uniform(-0.0009, 0.0009)
                            ws_dis = ws + n
                            if ws_dis>w: # check if disturbed sum is larger than original sum
                                p = p+1
                        p_value= p/1000
                        #p_value=1
                        # Calculate pindep
                        pindep = np.nansum(hwi_series) / len(hwi_series) * np.nansum(spei12_series) / len(spei12_series)
                        #print(pindep)
                        #print(pactual)
                        lmf_point = pactual / pindep
                        lmf_array[i, j] = lmf_point
                        # Check if it is significant
                        if p_value < 0.05:  # 0.05 for 5% significance
                            ns_array[i, j] = 1
                            print('reject the null hypothesis that the joint exceedance between hazard X and hazard Y was introduced by random chance, not statistally significant at 5%')
            pbar.update()
    return lmf_array, ns_array


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from cartopy.io import shapereader
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
import geopandas as gpd
from tqdm.notebook import tqdm
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap 
def remove_leap_years(input_array, start_year):
    years = np.arange(start_year, start_year + 100)
    # Define the list of leap years
    leap_years_list = [1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
    # Find the indices corresponding to leap years in the years array
    leap_year_indices = np.where(np.isin(years, leap_years_list))[0]
    # Initialize the output array to store data without leap years
    output_array = []
    idx_start = 0
    idx_end = 28
    for i, year in enumerate(years):
        # Skip data for leap years
        output_array.append(input_array[idx_start:idx_end,:])
        if i in leap_year_indices:
            idx_start += 29
            idx_end += 29
        else:
            idx_start += 28
            idx_end += 28
    # Convert the list of arrays to a single NumPy array with shape (2800, 100, 100)
    output_array = np.concatenate(output_array, axis=0)
    return output_array
def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]
class MidpointNormalize(mcolors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mcolors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        v_ext = np.max( [ np.abs(self.vmin), np.abs(self.vmax) ] )
        x, y = [-v_ext, self.midpoint, v_ext], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

def plot_lmf(hazard, hwi, spei12, dfi):
    sy=int(hwi.time.values[0].astype('M8[Y]').astype(str))
    # Define the custom colorbar intervals
    custom_ticks = [0, 1, 2]
    # Create a custom color map
    minimum=0
    maximum=2
    midpoint = 1
    val = midpoint / maximum
    cmap = LinearSegmentedColormap.from_list(
        "mycolormap",
        colors=[
        (0.0, [0.0, 0.0, 0.5]),  # Dark blue
        (0.25, [0.6, 0.8, 1.0]),  # Light blue
        (0.45, [0.7, 1.0, 0.7]),  # Light green
        (0.50, [0.0, 0.5, 0.0]),  # Green
        (0.55, [1.0, 0.7, 0.7]),  # Light red
        (0.80, [1.0, 0.0, 0.0]),  # Red
        (1.0, [0.5, 0.0, 0.0])  # Dark red
            ]
    )
    #norm = mcolors.BoundaryNorm(custom_ticks, len(custom_ticks) - 1)
    # Define a grayscale colormap
    cmap_colors = [(0.5, 0.5, 0.5), (0.5, 0.5, 0.5)]  # Grey color for not significant values
    gray_cmap = ListedColormap(cmap_colors, name='CustomGrey')
    months = range(1, 13)
    # Set up the figure and axes
    fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(16, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    # Flatten the axes array for easier iteration
    axes = axes.flatten()
    # Loop over the axes and customize each subplot
    with tqdm(total=len(months)) as pbar:
        for i, ax in enumerate(axes):
            month = months[i]
            # Select data for the specific month
            hwi_month = hwi.sel(time=hwi['time'].dt.month == month)
            spei12_month = spei12.sel(time=spei12['time'].dt.month == month)
            dfi_month = dfi_flood_index.where(dfi['time'].dt.month == month, drop=True)
            hwi_array = hwi_month['heatwave index'].values
            spei12_array = spei12_month['SPEI'].values
            dfi_array = dfi_month.values
            if i==0:
                # Create a grid of longitudes and latitudes
                lon_grid, lat_grid = np.meshgrid(hwi_month['lon'], hwi_month['lat'])
                # Add the country boundaries
                resolution = '10m'
                category = 'cultural'
                name = 'admin_0_countries'
                shpfilename = shapereader.natural_earth(resolution, category, name)
                df = gpd.read_file(shpfilename)
                # get geometry of a country
                poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]
                stamen_terrain = cimgt.Stamen('terrain-background')
                pad1 = .1  # padding, degrees unit
                exts = [poly[0].bounds[0] - pad1, poly[0].bounds[2] + pad1, poly[0].bounds[1] - pad1,
                        poly[0].bounds[3] + pad1];
                # make a mask polygon by polygon's difference operation
                msk = Polygon(rect_from_bound(*exts)).difference(poly[0].simplify(0.01))
                msk_stm = ccrs.PlateCarree().project_geometry(msk, ccrs.PlateCarree())  # project geometry to the projection used by stamen
            ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black')
            ax.set_extent(exts, crs=ccrs.PlateCarree())
            # plot the mask using semi-transparency (alpha=0.65) on the masked-out portion
            ax.add_geometries(msk_stm, ccrs.PlateCarree(), zorder=12, facecolor='white', edgecolor='k', alpha=1)
            # Plot the contour lines for the correlation between Hazards
            if hazard[0:21] == 'heat wave and drought':
                lmf_array, ns_array=lmf(hwi_array, spei12_array, 0, -1, 'lower', 'upper')
                #print(np.nanmin(lmf_array), np.nanmax(lmf_array))
                im = ax.imshow(lmf_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=0, vmax=2)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:19] == 'heat wave and flood':
                if month==2:
                    hwi_array = remove_leap_years(hwi_array, sy)
                lmf_array, ns_array=lmf(hwi_array, dfi_array, 0, 0, 'lower', 'lower')
                im = ax.imshow(lmf_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=0, vmax=2)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:17] == 'drought and flood':
                if month==2:
                    spei12_array = remove_leap_years(spei12_array, sy)
                lmf_array, ns_array=lmf(dfi_array, spei12_array, 0, -1, 'lower', 'upper')
                im = ax.imshow(lmf_array,
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=0, vmax=2)
                im2 = ax.imshow(ns_array,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            # Add a title
            month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                          'October', 'November', 'December']
            ax.set_title(month_list[month - 1], fontsize=20)
            pbar.update()
    cbar_ax = fig.add_axes([0, 0, 1, 0.05])  # [left, bottom, width, height]
    # Create the colorbar using the custom ticks and boundaries
    # Create a custom colormap using ListedColormap
    #colors = plt.cm.coolwarm_r(norm(np.arange(len(custom_ticks) - 1)))
    cbar = plt.colorbar(im, cax=cbar_ax, ticks=custom_ticks, cmap=cmap, 
                        orientation='horizontal', extend='max')
    #cbar.set_ticklabels(['0', '1', '10'])  # Set tick labels
    #cbar.set_ticks(custom_ticks)  # Set tick positions
    cbar.ax.tick_params(labelsize=20)
    label = 'Likelihood multiplication factor: ' + hazard
    cbar.set_label(label, fontsize=20)
    plt.subplots_adjust(wspace=0.2, hspace=-0.2)  # Adjust the value of hspace as needed to reduce the space between rows
    plt.tight_layout()
    # Show the plot
    plt.show()


In [ ]:
plot_lmf('heat wave and drought', hwi, spei12, dfi)

In [ ]:
plot_lmf('heat wave and flood', hwi, spei12, dfi)

In [ ]:
plot_lmf('drought and flood', hwi, spei12, dfi) 

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from cartopy.io import shapereader
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
import geopandas as gpd
from tqdm.notebook import tqdm
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap 
def lmf_change(lmf_array_1, lmf_array_2):
    # Assuming lmf_array_1 and lmf_array_2 have the same shape
    # Calculate the relative change as a percentage
    lmf_array_change = ((lmf_array_2 - lmf_array_1) / lmf_array_1) * 100
    # If you want to handle cases where the denominator (lmf_array_1) is zero (NaN after replacement),
    # you can set those elements in lmf_array_change to NaN
    lmf_array_change[np.isnan(lmf_array_1)] = np.nan
    return lmf_array_change
def remove_leap_years(input_array, start_year):
    years = np.arange(start_year, start_year + 100)
    # Define the list of leap years
    leap_years_list = [1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
    # Find the indices corresponding to leap years in the years array
    leap_year_indices = np.where(np.isin(years, leap_years_list))[0]
    # Initialize the output array to store data without leap years
    output_array = []
    idx_start = 0
    idx_end = 28
    for i, year in enumerate(years):
        # Skip data for leap years
        output_array.append(input_array[idx_start:idx_end,:])
        if i in leap_year_indices:
            idx_start += 29
            idx_end += 29
        else:
            idx_start += 28
            idx_end += 28
    # Convert the list of arrays to a single NumPy array with shape (2800, 100, 100)
    output_array = np.concatenate(output_array, axis=0)
    return output_array
def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]
class MidpointNormalize(mcolors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mcolors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        v_ext = np.max( [ np.abs(self.vmin), np.abs(self.vmax) ] )
        x, y = [-v_ext, self.midpoint, v_ext], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

def plot_lmf_diff(hazard, hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2):
    # Define the custom colorbar intervals
    custom_ticks = [-100, -80, -60, -40, -20, 0, 20, 40, 60, 80, 100]
    # Create a custom color map
    minimum=-100
    maximum=100
    midpoint = 0
    cmap = 'coolwarm'
    # Define a grayscale colormap
    cmap_colors = [(0.5, 0.5, 0.5), (0.5, 0.5, 0.5)]  # Grey color for not significant values
    gray_cmap = ListedColormap(cmap_colors, name='CustomGrey')
    months = range(1, 13)
    # Set up the figure and axes
    fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(16, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    # Flatten the axes array for easier iteration
    axes = axes.flatten()
    # Loop over the axes and customize each subplot
    with tqdm(total=len(months)) as pbar:
        for i, ax in enumerate(axes):
            month = months[i]
            # Select data for the specific month
            hwi_month_1 = hwi_1.sel(time=hwi_1['time'].dt.month == month)
            spei12_month_1 = spei12_1.sel(time=spei12_1['time'].dt.month == month)
            dfi_flood_index = dfi_1['flood_index']
            dfi_month_1 = dfi_flood_index.where(dfi_1['time'].dt.month == month, drop=True)
            hwi_array_1 = hwi_month_1['heatwave index'].values
            spei12_array_1 = spei12_month_1['SPEI'].values
            dfi_array_1 = dfi_month_1.values
            
            hwi_month_2 = hwi_2.sel(time=hwi_2['time'].dt.month == month)
            spei12_month_2 = spei12_2.sel(time=spei12_2['time'].dt.month == month)
            dfi_flood_index = dfi_2['flood_index']
            dfi_month_2 = dfi_flood_index.where(dfi_2['time'].dt.month == month, drop=True)
            hwi_array_2 = hwi_month_2['heatwave index'].values
            spei12_array_2 = spei12_month_2['SPEI'].values
            dfi_array_2 = dfi_month_2.values
            if i==0:
                # Create a grid of longitudes and latitudes
                lon_grid, lat_grid = np.meshgrid(hwi_month_1['lon'], hwi_month_1['lat'])
                # Add the country boundaries
                resolution = '10m'
                category = 'cultural'
                name = 'admin_0_countries'
                shpfilename = shapereader.natural_earth(resolution, category, name)
                df = gpd.read_file(shpfilename)
                # get geometry of a country
                poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]
                stamen_terrain = cimgt.Stamen('terrain-background')
                pad1 = .1  # padding, degrees unit
                exts = [poly[0].bounds[0] - pad1, poly[0].bounds[2] + pad1, poly[0].bounds[1] - pad1,
                        poly[0].bounds[3] + pad1];
                # make a mask polygon by polygon's difference operation
                msk = Polygon(rect_from_bound(*exts)).difference(poly[0].simplify(0.01))
                msk_stm = ccrs.PlateCarree().project_geometry(msk, ccrs.PlateCarree())  # project geometry to the projection used by stamen
            ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black')
            ax.set_extent(exts, crs=ccrs.PlateCarree())
            # plot the mask using semi-transparency (alpha=0.65) on the masked-out portion
            ax.add_geometries(msk_stm, ccrs.PlateCarree(), zorder=12, facecolor='white', edgecolor='k', alpha=1)
            # Plot the contour lines for the correlation between Hazards
            if hazard[0:21] == 'heat wave and drought':
                lmf_array_1, ns_array_1=lmf(hwi_array_1, spei12_array_1, 0, -1, 'lower', 'upper')
                lmf_array_2, ns_array_2=lmf(hwi_array_2, spei12_array_2, 0, -1, 'lower', 'upper')
                #print(np.nanmin(lmf_array), np.nanmax(lmf_array))
                im = ax.imshow(lmf_change(lmf_array_1, lmf_array_2),
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=-100, vmax=100)
                im2 = ax.imshow(ns_array_1,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
                im3 = ax.imshow(ns_array_2,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:19] == 'heat wave and flood':
                if month==2:
                    hwi_array_1 = remove_leap_years(hwi_array_1, 1922)
                    hwi_array_2 = remove_leap_years(hwi_array_2, 1972)
                lmf_array_1, ns_array_1=lmf(hwi_array_1, dfi_array_1, 0, 0, 'lower', 'lower')
                lmf_array_2, ns_array_2=lmf(hwi_array_2, dfi_array_2, 0, 0, 'lower', 'lower')
                im = ax.imshow(lmf_change(lmf_array_1, lmf_array_2),
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=-100, vmax=100)
                im2 = ax.imshow(ns_array_1,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
                im3 = ax.imshow(ns_array_2,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:17] == 'drought and flood':
                if month==2:
                    spei12_array_1 = remove_leap_years(spei12_array_1, 1922)
                    spei12_array_2 = remove_leap_years(spei12_array_2, 1972)
                lmf_array_1, ns_array_1=lmf(dfi_array_1, spei12_array_1, 0, -1, 'lower', 'upper')
                lmf_array_2, ns_array_2=lmf(dfi_array_2, spei12_array_2, 0, -1, 'lower', 'upper')
                im = ax.imshow(lmf_change(lmf_array_1, lmf_array_2),
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=-100, vmax=100)
                im2 = ax.imshow(ns_array_1,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
                im3 = ax.imshow(ns_array_2,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            # Add a title
            month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                          'October', 'November', 'December']
            ax.set_title(month_list[month - 1], fontsize=20)
            pbar.update()
    cbar_ax = fig.add_axes([0, 0, 1, 0.05])  # [left, bottom, width, height]
    # Create the colorbar using the custom ticks and boundaries
    cbar = plt.colorbar(im, cax=cbar_ax, ticks=custom_ticks, cmap=cmap, 
                        orientation='horizontal', extend='both')
    cbar.ax.tick_params(labelsize=20)
    label = 'Relative Change of LMF Between 1972-2021 and 1922-1971 (%): ' + hazard
    cbar.set_label(label, fontsize=20)
    plt.subplots_adjust(wspace=0.2, hspace=-0.2)  # Adjust the value of hspace as needed to reduce the space between rows
    plt.tight_layout()
    # Show the plot
    plt.show()

In [ ]:
# Calculate change of return periods from 1922-1971 and 1972-2021
start_date1 = pd.to_datetime('1922-01-31')
end_date1 = pd.to_datetime('1971-12-31')

# Truncate hwi array
hwi_1 = hwi.sel(time=slice(start_date1, end_date1))

# Truncate spei12 array
spei12_1 = spei12.sel(time=slice(start_date1, end_date1))

# Truncate dfi array
dfi_1 = dfi.sel(time=slice(start_date1, end_date1))

start_date2 = pd.to_datetime('1972-01-01')
end_date2 = pd.to_datetime('2021-12-31')

# Truncate hwi array
hwi_2 = hwi.sel(time=slice(start_date2, end_date2))

# Truncate spei12 array
spei12_2 = spei12.sel(time=slice(start_date2, end_date2))

# Truncate dfi array
dfi_2 = dfi.sel(time=slice(start_date2, end_date2))

In [ ]:
plot_lmf('heat wave and drought (1922-1971)', hwi_1, spei12_1, dfi_1)

In [ ]:
plot_lmf('heat wave and drought (1972-2021)', hwi_2, spei12_2, dfi_2)

In [ ]:
plot_lmf_diff('heat wave and drought', hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2)

In [ ]:
plot_lmf_diff('drought and flood', hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2)

In [ ]:
plot_lmf_diff('heat wave and flood', hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2)

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from cartopy.io import shapereader
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
import geopandas as gpd
from tqdm.notebook import tqdm
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap 
def rp_change(rp_array_1, rp_array_2):
    # Calculate the absolute change 
    rp_array_change = (rp_array_2 - rp_array_1)
    return rp_array_change
def remove_leap_years(input_array, start_year):
    years = np.arange(start_year, start_year + 100)
    # Define the list of leap years
    leap_years_list = [1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
    # Find the indices corresponding to leap years in the years array
    leap_year_indices = np.where(np.isin(years, leap_years_list))[0]
    # Initialize the output array to store data without leap years
    output_array = []
    idx_start = 0
    idx_end = 28
    for i, year in enumerate(years):
        # Skip data for leap years
        output_array.append(input_array[idx_start:idx_end,:])
        if i in leap_year_indices:
            idx_start += 29
            idx_end += 29
        else:
            idx_start += 28
            idx_end += 28
    # Convert the list of arrays to a single NumPy array with shape (2800, 100, 100)
    output_array = np.concatenate(output_array, axis=0)
    return output_array
def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]
class MidpointNormalize(mcolors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mcolors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        v_ext = np.max( [ np.abs(self.vmin), np.abs(self.vmax) ] )
        x, y = [-v_ext, self.midpoint, v_ext], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

def plot_rp_diff(hazard, hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2):
    # Define the custom colorbar intervals
    custom_ticks = [-100, -80, -60, -40, -20, 0, 20, 40, 60, 80, 100]
    minimum=-100
    maximum=100
    midpoint = 0
    cmap = 'coolwarm_r'
    #norm = mcolors.BoundaryNorm(custom_ticks, len(custom_ticks) - 1)
    # Define a grayscale colormap
    cmap_colors = [(0.5, 0.5, 0.5), (0.5, 0.5, 0.5)]  # Grey color for not significant values
    gray_cmap = ListedColormap(cmap_colors, name='CustomGrey')
    months = range(1, 13)
    # Set up the figure and axes
    fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(16, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    # Flatten the axes array for easier iteration
    axes = axes.flatten()
    # Loop over the axes and customize each subplot
    with tqdm(total=len(months)) as pbar:
        for i, ax in enumerate(axes):
            month = months[i]
            # Select data for the specific month
            hwi_month_1 = hwi_1.sel(time=hwi_1['time'].dt.month == month)
            spei12_month_1 = spei12_1.sel(time=spei12_1['time'].dt.month == month)
            dfi_flood_index = dfi_1['flood_index']
            dfi_month_1 = dfi_flood_index.where(dfi_1['time'].dt.month == month, drop=True)
            hwi_array_1 = hwi_month_1['heatwave index'].values
            spei12_array_1 = spei12_month_1['SPEI'].values
            dfi_array_1 = dfi_month_1.values
            
            hwi_month_2 = hwi_2.sel(time=hwi_2['time'].dt.month == month)
            spei12_month_2 = spei12_2.sel(time=spei12_2['time'].dt.month == month)
            dfi_flood_index = dfi_2['flood_index']
            dfi_month_2 = dfi_flood_index.where(dfi_2['time'].dt.month == month, drop=True)
            hwi_array_2 = hwi_month_2['heatwave index'].values
            spei12_array_2 = spei12_month_2['SPEI'].values
            dfi_array_2 = dfi_month_2.values
            if i==0:
                # Create a grid of longitudes and latitudes
                lon_grid, lat_grid = np.meshgrid(hwi_month_1['lon'], hwi_month_1['lat'])
                # Add the country boundaries
                resolution = '10m'
                category = 'cultural'
                name = 'admin_0_countries'
                shpfilename = shapereader.natural_earth(resolution, category, name)
                df = gpd.read_file(shpfilename)
                # get geometry of a country
                poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]
                stamen_terrain = cimgt.Stamen('terrain-background')
                pad1 = .1  # padding, degrees unit
                exts = [poly[0].bounds[0] - pad1, poly[0].bounds[2] + pad1, poly[0].bounds[1] - pad1,
                        poly[0].bounds[3] + pad1];
                # make a mask polygon by polygon's difference operation
                msk = Polygon(rect_from_bound(*exts)).difference(poly[0].simplify(0.01))
                msk_stm = ccrs.PlateCarree().project_geometry(msk, ccrs.PlateCarree())  # project geometry to the projection used by stamen
            ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black')
            ax.set_extent(exts, crs=ccrs.PlateCarree())
            # plot the mask using semi-transparency (alpha=0.65) on the masked-out portion
            ax.add_geometries(msk_stm, ccrs.PlateCarree(), zorder=12, facecolor='white', edgecolor='k', alpha=1)
            # Plot the contour lines for the correlation between Hazards
            if hazard[0:21] == 'heat wave and drought':
                rp_array_1, ns_array_1=rp(hwi_array_1, spei12_array_1, 0, -1, 'lower', 'upper')
                rp_array_2, ns_array_2=rp(hwi_array_2, spei12_array_2, 0, -1, 'lower', 'upper')
                #print(np.nanmin(rp_array), np.nanmax(rp_array))
                im = ax.imshow(rp_change(rp_array_1, rp_array_2),
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=-100, vmax=100)
                im2 = ax.imshow(ns_array_1,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
                im3 = ax.imshow(ns_array_2,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:19] == 'heat wave and flood':
                if month==2:
                    hwi_array_1 = remove_leap_years(hwi_array_1, 1922)
                    hwi_array_2 = remove_leap_years(hwi_array_2, 1972)
                rp_array_1, ns_array_1=rp(hwi_array_1, dfi_array_1, 0, 0, 'lower', 'lower')
                rp_array_2, ns_array_2=rp(hwi_array_2, dfi_array_2, 0, 0, 'lower', 'lower')
                im = ax.imshow(rp_change(rp_array_1, rp_array_2),
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=-100, vmax=100)
                im2 = ax.imshow(ns_array_1,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
                im3 = ax.imshow(ns_array_2,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            elif hazard[0:17] == 'drought and flood':
                if month==2:
                    spei12_array_1 = remove_leap_years(spei12_array_1, 1922)
                    spei12_array_2 = remove_leap_years(spei12_array_2, 1972)
                rp_array_1, ns_array_1=rp(dfi_array_1, spei12_array_1, 0, -1, 'lower', 'upper')
                rp_array_2, ns_array_2=rp(dfi_array_2, spei12_array_2, 0, -1, 'lower', 'upper')
                im = ax.imshow(rp_change(rp_array_1, rp_array_2),
                            origin='lower',
                            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                            cmap=cmap, zorder=11, vmin=-100, vmax=100)
                im2 = ax.imshow(ns_array_1,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
                im3 = ax.imshow(ns_array_2,
                           origin='lower',
                           extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                           cmap=gray_cmap, zorder=11)
            # Add a title
            month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                          'October', 'November', 'December']
            ax.set_title(month_list[month - 1], fontsize=20)
            pbar.update()
    cbar_ax = fig.add_axes([0, 0, 1, 0.05])  # [left, bottom, width, height]
    # Create the colorbar using the custom ticks and boundaries
    # Create a custom colormap using ListedColormap
    #colors = plt.cm.coolwarm_r(norm(np.arange(len(custom_ticks) - 1)))
    cbar = plt.colorbar(im, cax=cbar_ax, ticks=custom_ticks, cmap=cmap, 
                        orientation='horizontal', extend='both')
    #cbar.set_ticklabels(['0', '1', '10'])  # Set tick labels
    #cbar.set_ticks(custom_ticks)  # Set tick positions
    cbar.ax.tick_params(labelsize=20)
    label = 'Change of Return Periods Between 1972-2021 and 1922-1971 (years): ' + hazard
    cbar.set_label(label, fontsize=20)
    plt.subplots_adjust(wspace=0.2, hspace=-0.2)  # Adjust the value of hspace as needed to reduce the space between rows
    plt.tight_layout()
    # Show the plot
    plt.show()

In [ ]:
plot_rp('heat wave and drought (1922-1971)', hwi_1, spei12_1, dfi_1)

In [ ]:
plot_rp('heat wave and drought (1972-2021)', hwi_2, spei12_2, dfi_2)

In [ ]:
plot_rp_diff('heat wave and drought', hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2)

In [ ]:
plot_rp_diff('drought and flood', hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2)

In [ ]:
plot_rp_diff('heat wave and flood', hwi_1, spei12_1, dfi_1, hwi_2, spei12_2, dfi_2)